In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os
import io

#### Data Cleaning
**Main Goals**
- Increase granularity: Get one observation per team per match
    - Each match has 2 rows, one for the stats for each team
    - add column: Home/away
- Remove data from before 2017


##### Cleaning passing_data to set it up for a merge into data_full df

**1. Match date format to the metrics csv**

In [11]:
passing_data = pd.read_csv("data/passing_data.csv")
passing_data['Date'] = pd.to_datetime(passing_data['Date']).dt.strftime('%d/%m/%y')
passing_data.head()

,Date,Team,Passing Cmp%
0,11/08/17,Arsenal,83.0
1,19/08/17,Arsenal,85.9
2,27/08/17,Arsenal,80.4
3,09/09/17,Arsenal,78.8
4,17/09/17,Arsenal,78.4


**2. I want to rename the column to Pass Completion, and add a percent to the end of each value**

In [12]:
passing_data.rename(columns={'Passing Cmp%': 'Pass Cmp'}, inplace=True)

#undo multiple percents from multiple runs
#passing_data['Pass Completion'] = passing_data['Pass Completion'].str.rstrip('%') + '%'
#prevent
passing_data['Pass Cmp'] = passing_data['Pass Cmp'].apply(lambda x: f"{x}%" if not str(x).endswith('%') else x)
passing_data.head(5)

,Date,Team,Pass Cmp
0,11/08/17,Arsenal,83.0%
1,19/08/17,Arsenal,85.9%
2,27/08/17,Arsenal,80.4%
3,09/09/17,Arsenal,78.8%
4,17/09/17,Arsenal,78.4%


**3.Create a copy dataframe of passing_data for safekeeping, match team names in passing_data to those in the metrics df**

In [13]:
passing_data_copy = passing_data.copy()

#typo lol
passing_data['Team'] = passing_data['Team'].replace({'Leiceste': 'Leicester'})

team_name_mapping = {
    'Manchester City': 'Man City',
    'Brighton & Hove Albion': 'Brighton',
    'Manchester United': 'Man United',
    'Newcastle United': 'Newcastle',
    'West Ham United': 'West Ham',
    'Wolverhampton Wanderers': 'Wolves'
}

passing_data['Team'] = passing_data['Team'].replace(team_name_mapping)
#print(passing_data['Team'].unique())
#passing_data.head(5)



**This looks better, this is now ready for the merge. I will do this later in the file when I know the data_full df is cleaned.**

In [14]:
dfs = []
for filename in os.listdir('data/Datasets'):
    if (filename == '.ipynb_checkpoints'):
        continue
    else:
        data_fp = Path('data/Datasets') / filename
        dfs.append(pd.read_csv(data_fp))
        data = pd.concat(dfs).reset_index(drop=True)
data.shape

(8020, 144)

In [15]:
to_drop = ['Referee', 
            'HTHG', # dropping half time stuff
            'HTAG', 
            'Div', 
            'HTR']
maybe_drop = ['Attendance',
                'HO',
                'AO', 'BWA', 'IWH', 'IWD', 'IWA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA',
        'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD', 'MaxA', 'AvgH', 'AvgD', 'AvgA',
        'B365>2.5', 'B365<2.5', 'P>2.5', 'P<2.5', 'Max>2.5', 'Max<2.5',
        'Avg>2.5', 'Avg<2.5', 'AHh', 'B365AHH', 'B365AHA', 'PAHH', 'PAHA',
        'MaxAHH', 'MaxAHA', 'AvgAHH', 'AvgAHA', 'B365CH', 'B365CD', 'B365CA',
        'BWCH', 'BWCD', 'BWCA', 'IWCH', 'IWCD', 'IWCA', 'PSCH', 'PSCD', 'PSCA',
        'WHCH', 'WHCD', 'WHCA', 'VCCH', 'VCCD', 'VCCA', 'MaxCH', 'MaxCD',
        'MaxCA', 'AvgCH', 'AvgCD', 'AvgCA', 'B365C>2.5', 'B365C<2.5', 'PC>2.5',
        'PC<2.5', 'MaxC>2.5', 'MaxC<2.5', 'AvgC>2.5', 'AvgC<2.5', 'AHCh',
        'B365CAHH', 'B365CAHA', 'PCAHH', 'PCAHA', 'MaxCAHH', 'MaxCAHA',
        'AvgCAHH', 'AvgCAHA', 'B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'Time', 'LBH',
       'LBD', 'LBA', 'Bb1X2', 'BbMxH', 'BbAvH', 'BbMxD', 'BbAvD', 'BbMxA',
       'BbAvA', 'BbOU', 'BbMx>2.5', 'BbAv>2.5', 'BbMx<2.5', 'BbAv<2.5', 'BbAH',
       'BbAHh', 'BbMxAHH', 'BbAvAHH', 'BbMxAHA', 'BbAvAHA', 'SJH', 'SJD',
       'SJA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA', 'HHW', 'AHW', 'SBH',
       'SBD', 'SBA']

data = data.drop(columns=to_drop)
for col in maybe_drop:
    try:
        data = data.drop(columns=[col])
    except KeyError:
        continue

In [16]:
data.columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HS', 'AS',
       'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR'],
      dtype='object')

AHW/HHW = Hit Woodwork: hit crossbar/goalpost

- NOT included in Shots on Target

It looks like all the columns that start with `H` are home, and the columns that start with `A` are away
- exceptions: FTHG = Home team goals, FTAG = Away team goals, HTHG = half time home goals
    - The date, time, div, etc. are also obviously the same for home and away

In [17]:
is_away = data.columns.str[0] == 'A'
is_home = data.columns.str[0] == 'H' # necessary, since there are exception
away_cols = data.columns[is_away]
home_cols = data.columns[is_home]
away_cols.size == home_cols.size

True

In [18]:
away_df = pd.concat(
    [
        data.loc[:,['Date', 'FTAG', 'FTR']],
        data[away_cols]
    ],
    axis=1
)
home_df = pd.concat(
    [
        data.loc[:,['Date', 'FTHG', 'FTR']],
        data[home_cols]
    ],
    axis=1
)

In [19]:
away_df.head()

,Date,FTAG,FTR,AwayTeam,AS,AST,AF,AC,AY,AR
0,13/08/11,2,A,Wolves,13,4,10,6,2,0
1,13/08/11,0,D,Aston Villa,7,1,18,3,4,0
2,13/08/11,1,D,Sunderland,15,6,12,3,4,0
3,13/08/11,0,D,Arsenal,9,4,11,5,5,1
4,13/08/11,4,A,Bolton,13,7,16,2,2,0


In [20]:
home_df.head()

,Date,FTHG,FTR,HomeTeam,HS,HST,HF,HC,HY,HR
0,13/08/11,1,A,Blackburn,16,8,14,12,4,0
1,13/08/11,0,D,Fulham,13,9,10,2,2,0
2,13/08/11,1,D,Liverpool,11,4,17,6,4,0
3,13/08/11,0,D,Newcastle,6,1,9,2,3,0
4,13/08/11,0,A,QPR,13,7,9,3,1,1


In [21]:
print(home_df.shape)
print(away_df.shape)
print(data.shape)

(8020, 10)
(8020, 10)
(8020, 18)


Let's add one final column to both dataframes: a column to tell if the team is home or away.

In [22]:
away_df['IsHome'] = [False] * away_df.shape[0]
away_df.head(3)

,Date,FTAG,FTR,AwayTeam,AS,AST,AF,AC,AY,AR,IsHome
0,13/08/11,2,A,Wolves,13,4,10,6,2,0,False
1,13/08/11,0,D,Aston Villa,7,1,18,3,4,0,False
2,13/08/11,1,D,Sunderland,15,6,12,3,4,0,False


In [23]:
home_df['IsHome'] = [True] * home_df.shape[0]
home_df.head(3)

,Date,FTHG,FTR,HomeTeam,HS,HST,HF,HC,HY,HR,IsHome
0,13/08/11,1,A,Blackburn,16,8,14,12,4,0,True
1,13/08/11,0,D,Fulham,13,9,10,2,2,0,True
2,13/08/11,1,D,Liverpool,11,4,17,6,4,0,True


Our ultimate goal is to run a `pd.concat()` on both to combine them into one df with twice the rows of the original. To combine these, they'll have to have the same column names.

Since the distinction for everything but the first few columns and '**FTHG/FTAG**' is the first letters, let's just drop it from every column name.
'AT' for away team, and 'HT' for home team.

In [24]:
away_df.head(3)

,Date,FTAG,FTR,AwayTeam,AS,AST,AF,AC,AY,AR,IsHome
0,13/08/11,2,A,Wolves,13,4,10,6,2,0,False
1,13/08/11,0,D,Aston Villa,7,1,18,3,4,0,False
2,13/08/11,1,D,Sunderland,15,6,12,3,4,0,False


In [25]:
away_df.columns

Index(['Date', 'FTAG', 'FTR', 'AwayTeam', 'AS', 'AST', 'AF', 'AC', 'AY', 'AR',
       'IsHome'],
      dtype='object')

In [26]:
# we'll work with away first
fixed_away_cols = (away_df
                   .columns
                   .str.replace('Away', '')
                   .str.replace('A', ''))
away_df.columns = fixed_away_cols
away_df.head(3)

,Date,FTG,FTR,Team,S,ST,F,C,Y,R,IsHome
0,13/08/11,2,A,Wolves,13,4,10,6,2,0,False
1,13/08/11,0,D,Aston Villa,7,1,18,3,4,0,False
2,13/08/11,1,D,Sunderland,15,6,12,3,4,0,False


In [27]:
home_df.columns

Index(['Date', 'FTHG', 'FTR', 'HomeTeam', 'HS', 'HST', 'HF', 'HC', 'HY', 'HR',
       'IsHome'],
      dtype='object')

In [28]:
# same thing with home
fixed_home_cols = (home_df
                   .columns
                   .str.replace('HomeTeam', 'Team') # just so we dont delete "IsHome"
                   .str.replace('FTHG', 'FTG')
                   .str.replace('HS', 'S')
                   .str.replace('HST', 'ST')
                   .str.replace('HHW', 'HW')
                   .str.replace('HC','C')
                   .str.replace('HF', 'F')
                   .str.replace('HY', 'Y')
                   .str.replace('HR', 'R'))
home_df.columns = fixed_home_cols
home_df.head(3)

,Date,FTG,FTR,Team,S,ST,F,C,Y,R,IsHome
0,13/08/11,1,A,Blackburn,16,8,14,12,4,0,True
1,13/08/11,0,D,Fulham,13,9,10,2,2,0,True
2,13/08/11,1,D,Liverpool,11,4,17,6,4,0,True


In [29]:
(home_df.columns == away_df.columns).all()

True

Yippee!! Let's concatenate them both into one dataframe, to make it easier to work with.

In [30]:
data_full = pd.concat([away_df, home_df])
data_full.shape
data_full.shape[0] == data.shape[0] * 2

True

We have some weird, undescriptive column names right now, so let's replace those

In [31]:
data_full.columns

Index(['Date', 'FTG', 'FTR', 'Team', 'S', 'ST', 'F', 'C', 'Y', 'R', 'IsHome'], dtype='object')

In [32]:
better_col_names = {
    'FTR': 'FullTimeResult',
    'FTG': 'TotalGoalsScored',
    'S': 'Shots',
    'ST': 'ShotsOnTarget',
    'HW': 'HitWoodwork',
    'C': 'Corners',
    'F': 'FoulsCommited',
    'Y': 'YellowCardsEarned',
    'R': 'RedCardsEarned'
}
data_full.rename(columns=better_col_names, inplace=True)
data_full.sort_values('TotalGoalsScored').head(10)

,Date,TotalGoalsScored,FullTimeResult,Team,Shots,ShotsOnTarget,FoulsCommited,Corners,YellowCardsEarned,RedCardsEarned,IsHome
6754,28/08/05,0,A,Middlesbrough,10,6,18,6,2,0,True
6455,03/11/2002,0,A,Fulham,5,4,10,10,1,0,True
6456,03/11/2002,0,A,Leeds,18,5,14,9,3,0,True
7922,29/09/18,0,H,Fulham,6,0,13,1,3,0,False
4025,19/01/08,0,H,Sunderland,14,8,13,7,2,0,False
6457,03/11/2002,0,D,Tottenham,15,7,17,8,3,0,True
4022,19/01/08,0,D,Bolton,7,5,15,6,2,0,False
7924,29/09/18,0,H,Brighton,4,1,10,3,3,0,False
4936,13/05/18,0,A,Southampton,8,3,8,1,3,0,True
4018,13/01/08,0,H,Portsmouth,9,5,6,5,1,0,False


Much better, and much easier to work with.

Let's add a column to see if the team won or not (draws are marked as NaN for now)

In [33]:
data_full['Won'] = data_full.apply(
    lambda row: np.NaN if row['FullTimeResult'] == 'D' else (row['FullTimeResult'] == 'H') == (row['IsHome']),
    axis=1
)
data_full.head()

,Date,TotalGoalsScored,FullTimeResult,Team,Shots,ShotsOnTarget,FoulsCommited,Corners,YellowCardsEarned,RedCardsEarned,IsHome,Won
0,13/08/11,2,A,Wolves,13,4,10,6,2,0,False,True
1,13/08/11,0,D,Aston Villa,7,1,18,3,4,0,False,NaN
2,13/08/11,1,D,Sunderland,15,6,12,3,4,0,False,NaN
3,13/08/11,0,D,Arsenal,9,4,11,5,5,1,False,NaN
4,13/08/11,4,A,Bolton,13,7,16,2,2,0,False,True


In [34]:
data_full['FullTimeResult'].value_counts()

FullTimeResult
H    7334
A    4662
D    4044
Name: count, dtype: int64

In [35]:
data_full.shape

(16040, 12)

In [36]:
after_2017 = pd.to_datetime(data_full['Date']).dt.year >= 2017
data_full = data_full[after_2017].copy()
# data_full = pd.merge(data_full, passing_data_df[['Date', 'Team', 'Pass Cmp']], 
#                      on=['Date', 'Team'], 
#                      how='left')

data_full

/tmp/ipykernel_203/4185992096.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  after_2017 = pd.to_datetime(data_full['Date']).dt.year >= 2017


,Date,TotalGoalsScored,FullTimeResult,Team,Shots,ShotsOnTarget,FoulsCommited,Corners,YellowCardsEarned,RedCardsEarned,IsHome,Won
1328,01/01/17,0,H,Crystal Palace,7,4,7,4,1,0,False,False
1329,01/01/17,4,A,Tottenham,19,6,6,3,0,0,False,True
1330,02/01/17,0,H,Southampton,12,3,14,8,1,0,False,False
1331,02/01/17,1,H,Burnley,11,3,11,8,3,0,False,False
1332,02/01/17,0,D,Leicester,10,4,9,2,0,0,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
8015,08/12/18,0,A,Leicester,11,3,12,6,3,0,True,False
8016,08/12/18,4,H,Man United,20,11,11,10,1,0,True,True
8017,08/12/18,3,H,West Ham,13,6,10,5,1,0,True,True
8018,09/12/18,1,A,Newcastle,12,4,10,4,2,1,True,False


### Managerial Changes dataset cleaning

In [37]:
managerial_changes = pd.read_csv('data/managerial_changes.csv')

In [38]:
managerial_changes.loc[managerial_changes['club'] == 'Manchester United']

,club,Coach,Matchday,Rank,PPG,last_match,last_match_teams_rank,Leaving date,Days in charge,Successor,Rank1,PPG1,first_match,year
225,Manchester United,Sir Alex Ferguson,38,1,2.34,West Bromwich Albion 5:5 Manchester United,5:5,"Jun 30, 2013",9733.0,David Moyes,-,-,West Bromwich Albion Manchester United,2012
239,Manchester United,David Moyes,35,7,1.68,Everton FC 2:0 Manchester United,2:0,"Apr 22, 2014",295.0,Louis van Gaal,-,-,Everton FC Manchester United,2013
276,Manchester United,Louis van Gaal,38,5,1.74,Crystal Palace 1:2 Manchester United,1:2,"May 23, 2016",679.0,José Mourinho,-,-,Crystal Palace Manchester United,2015
326,Manchester United,Ole Gunnar Solskjær,30,6,2.46,Wolverhampton Wanderers 2:1 Manchester United,2:1,"Mar 28, 2019",99.0,Ole Gunnar Solskjær,6,1.00,Wolverhampton Wanderers 1:5 Manchester United,2018
333,Manchester United,José Mourinho,17,6,1.53,Liverpool FC 3:1 Manchester United,3:1,"Dec 18, 2018",900.0,Ole Gunnar Solskjær,6,2.46,Liverpool FC 1:5 Manchester United,2018
362,Manchester United,Ralf Rangnick,38,6,1.54,Crystal Palace 1:0 Manchester United,1:0,"May 31, 2022",179.0,Erik ten Hag,-,-,Crystal Palace Manchester United,2021
368,Manchester United,Michael Carrick,14,7,2.00,Manchester United 3:2 Arsenal FC,3:2,"Dec 3, 2021",12.0,Ralf Rangnick,6,1.54,Manchester United 1:0 Arsenal FC,2021
369,Manchester United,Ole Gunnar Solskjær,12,8,1.42,Watford FC 4:1 Manchester United,4:1,"Nov 21, 2021",969.0,Michael Carrick,7,2.00,Watford FC 0:2 Manchester United,2021


In [39]:
managerial_changes.shape

(410, 14)

In [40]:
managerial_changes[['club', 'Coach', 'Leaving date', 'Successor']]

,club,Coach,Leaving date,Successor
0,Chelsea FC,David Webb,"Jun 30, 1993",Glenn Hoddle
1,Tottenham Hotspur,Doug Livermore,"Jun 19, 1993",Osvaldo Ardiles
2,Tottenham Hotspur,Ray Clemence,"Jun 19, 1993",Osvaldo Ardiles
3,Crystal Palace,Steve Coppell,"May 17, 1993",Alan Smith
4,Nottingham Forest,Brian Clough,"May 8, 1993",Frank Clark
...,...,...,...,...
405,Wolverhampton Wanderers,Bruno Lage,"Oct 2, 2022",Steve Davis
406,Brighton & Hove Albion,Andrew Crofts,"Sep 17, 2022",Roberto De Zerbi
407,Chelsea FC,Thomas Tuchel,"Sep 7, 2022",Graham Potter
408,Brighton & Hove Albion,Graham Potter,"Sep 7, 2022",Andrew Crofts


In [41]:
managers_cleaned = managerial_changes[['club', 'Coach', 'Leaving date', 'Successor']].copy()
managers_cleaned.columns = ['Club', 'Predecessor', 'Join date', 'Coach']
managers_cleaned[managers_cleaned['Club'] == 'Manchester United']

,Club,Predecessor,Join date,Coach
225,Manchester United,Sir Alex Ferguson,"Jun 30, 2013",David Moyes
239,Manchester United,David Moyes,"Apr 22, 2014",Louis van Gaal
276,Manchester United,Louis van Gaal,"May 23, 2016",José Mourinho
326,Manchester United,Ole Gunnar Solskjær,"Mar 28, 2019",Ole Gunnar Solskjær
333,Manchester United,José Mourinho,"Dec 18, 2018",Ole Gunnar Solskjær
362,Manchester United,Ralf Rangnick,"May 31, 2022",Erik ten Hag
368,Manchester United,Michael Carrick,"Dec 3, 2021",Ralf Rangnick
369,Manchester United,Ole Gunnar Solskjær,"Nov 21, 2021",Michael Carrick


Let's look through the data and do some checks for null values, strange values, etc.
By looking through our csv, it looks like null values are represented by '-', so we should catch that.

In [42]:
managers_cleaned = managers_cleaned.replace('-', np.NaN)

Now lets handle the missing values.

In [43]:
managers_cleaned.isna().sum()

Club           0
Predecessor    0
Join date      1
Coach          0
dtype: int64

In [44]:
# only one missing value - let's just drop it
managers_cleaned[managers_cleaned['Join date'].isna()]

,Club,Predecessor,Join date,Coach
62,Crystal Palace,Ron Noades,NaN,Jack Tresadern


In [45]:
managers_cleaned = managers_cleaned.drop(62, axis=0).reset_index(drop=True)

One problem I've noticed is that a lot of the team names don't line up between this and the metrics dataset. Let's fix that so we can more easily merge them later on.

In [46]:
np.sort(data_full['Team'].unique())

array(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton',
       'Burnley', 'Cardiff', 'Chelsea', 'Crystal Palace', 'Everton',
       'Fulham', 'Huddersfield', 'Hull', 'Leeds', 'Leicester',
       'Liverpool', 'Man City', 'Man United', 'Middlesbrough',
       'Newcastle', 'Norwich', 'Sheffield United', 'Southampton', 'Stoke',
       'Sunderland', 'Swansea', 'Tottenham', 'Watford', 'West Brom',
       'West Ham', 'Wolves'], dtype=object)

In [47]:
np.sort(managers_cleaned['Club'].unique())

array(['AFC Bournemouth', 'Arsenal FC', 'Aston Villa', 'Birmingham City',
       'Blackburn Rovers', 'Bolton Wanderers', 'Bradford City',
       'Brighton & Hove Albion', 'Burnley FC', 'Cardiff City',
       'Charlton Athletic', 'Chelsea FC', 'Coventry City',
       'Crystal Palace', 'Derby County', 'Everton FC', 'Fulham FC',
       'Huddersfield Town', 'Hull City', 'Ipswich Town', 'Leeds United',
       'Leicester City', 'Liverpool FC', 'Manchester City',
       'Manchester United', 'Middlesbrough FC', 'Newcastle United',
       'Norwich City', 'Nottingham Forest', 'Portsmouth FC',
       'Queens Park Rangers', 'Reading FC', 'Sheffield United',
       'Sheffield Wednesday', 'Southampton FC', 'Stoke City',
       'Sunderland AFC', 'Swansea City', 'Swindon Town',
       'Tottenham Hotspur', 'Watford FC', 'West Bromwich Albion',
       'West Ham United', 'Wigan Athletic', 'Wimbledon FC (- 2004)',
       'Wolverhampton Wanderers'], dtype=object)

In [48]:
cleaned_clubs = (managers_cleaned['Club']
                            .str.replace(' FC', '') # Step 1: no "FC" in metrics data
                            .str.replace('AFC', '') # Special case for "AFC Bournemouth"
                            .str.replace('Manchester City', 'Man *City') # Special case for man city
                            .str.replace(' City', '') # Step 2: no "City" in metrics data
                            .str.replace(' Rovers', '') # etc for all other suffixes
                            .str.replace(' Wanderers', '')
                            .str.replace(' & Hove Albion', '')
                            .str.replace(' Athletic', '')
                            .str.replace(' County', '')
                            .str.replace(' Town', '')
                            .str.replace('Manchester United', 'Man *United') # special case for man united
                            .str.replace(' United', '')
                            .str.replace(' A', '')
                            .str.replace(' Hotspur', '')
                            .str.replace('wichlbion', '')
                            .str.replace('Queens Park Rangers', 'QPR')
                            .str.replace('Wolverhampton', 'Wolves')
                            .str.replace('*', '', regex=False) # revert special cases
                            .str.strip())
# check for mismatched names
mc_clubs = np.sort(cleaned_clubs.unique())
metrics_clubs = np.sort(data_full['Team'].unique())
[x for x in [(mc, [club for club in metrics_clubs if club in mc]) for mc in mc_clubs] if not x[1] or x[0] != x[1][0]]

[('Birmingham', []),
 ('Blackburn', []),
 ('Bolton', []),
 ('Bradford', []),
 ('Charlton', []),
 ('Coventry', []),
 ('Derby', []),
 ('Ipswich', []),
 ('Nottingham Forest', []),
 ('Portsmouth', []),
 ('QPR', []),
 ('Reading', []),
 ('Sheffield', []),
 ('Sheffield Wednesday', []),
 ('Swindon', []),
 ('Wigan', []),
 ('Wimbledon (- 2004)', [])]

In [49]:
np.sort(data_full['Team'].unique())

array(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford', 'Brighton',
       'Burnley', 'Cardiff', 'Chelsea', 'Crystal Palace', 'Everton',
       'Fulham', 'Huddersfield', 'Hull', 'Leeds', 'Leicester',
       'Liverpool', 'Man City', 'Man United', 'Middlesbrough',
       'Newcastle', 'Norwich', 'Sheffield United', 'Southampton', 'Stoke',
       'Sunderland', 'Swansea', 'Tottenham', 'Watford', 'West Brom',
       'West Ham', 'Wolves'], dtype=object)

In [50]:
np.sort(cleaned_clubs.unique())

array(['Arsenal', 'Aston Villa', 'Birmingham', 'Blackburn', 'Bolton',
       'Bournemouth', 'Bradford', 'Brighton', 'Burnley', 'Cardiff',
       'Charlton', 'Chelsea', 'Coventry', 'Crystal Palace', 'Derby',
       'Everton', 'Fulham', 'Huddersfield', 'Hull', 'Ipswich', 'Leeds',
       'Leicester', 'Liverpool', 'Man City', 'Man United',
       'Middlesbrough', 'Newcastle', 'Norwich', 'Nottingham Forest',
       'Portsmouth', 'QPR', 'Reading', 'Sheffield', 'Sheffield Wednesday',
       'Southampton', 'Stoke', 'Sunderland', 'Swansea', 'Swindon',
       'Tottenham', 'Watford', 'West Brom', 'West Ham', 'Wigan',
       'Wimbledon (- 2004)', 'Wolves'], dtype=object)

Now the team names are looking a lot cleaner than before. Much more ready to merge.

In [51]:
managers_cleaned['Club'] = cleaned_clubs

In [52]:
# Here's the teams we're missing data for in one of the two dataframes:
set(data_full['Team']).difference(set(managers_cleaned['Club']))

{'Brentford', 'Sheffield United'}

Finally, let's change the date to something with similar formatting to our metrics data, and add the pass completeion stats to the dataframe. We need to eliminate the incosistency in date format to ensure a proper addition of this pass completion column.

In [53]:
np.random.choice(data_full['Date'].unique(), size=10)
data_full['Date'] = pd.to_datetime(data_full['Date'], dayfirst=True).dt.strftime('%d/%m/%y')

data_full = pd.merge(data_full, passing_data[['Date', 'Team', 'Pass Cmp']], 
                     on=['Date', 'Team'], 
                     how='left')

if 'Pass Cmp_x' in data_full.columns and 'Pass Cmp_y' in data_full.columns:
    data_full.drop(columns=['Pass Cmp_x', 'Pass Cmp_y'], inplace=True)

# Check the position of 'Pass Cmp' and move it if necessary
columns = list(data_full.columns)
if 'Pass Cmp' in columns and columns.index('Pass Cmp') != 6:  # 7th position (index starts from 0)
    pass_cmp_idx = columns.index('Pass Cmp')
    columns.insert(6, columns.pop(pass_cmp_idx))
    data_full = data_full[columns]
    
data_full

/tmp/ipykernel_203/3862339549.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_full['Date'] = pd.to_datetime(data_full['Date'], dayfirst=True).dt.strftime('%d/%m/%y')


,Date,TotalGoalsScored,FullTimeResult,Team,Shots,ShotsOnTarget,Pass Cmp,FoulsCommited,Corners,YellowCardsEarned,RedCardsEarned,IsHome,Won
0,01/01/17,0,H,Crystal Palace,7,4,NaN,7,4,1,0,False,False
1,01/01/17,4,A,Tottenham,19,6,NaN,6,3,0,0,False,True
2,02/01/17,0,H,Southampton,12,3,NaN,14,8,1,0,False,False
3,02/01/17,1,H,Burnley,11,3,NaN,11,8,3,0,False,False
4,02/01/17,0,D,Leicester,10,4,NaN,9,2,0,0,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,08/12/18,0,A,Leicester,11,3,76.2%,12,6,3,0,True,False
3500,08/12/18,4,H,Man United,20,11,83.4%,11,10,1,0,True,True
3501,08/12/18,3,H,West Ham,13,6,78.1%,10,5,1,0,True,True
3502,09/12/18,1,A,Newcastle,12,4,76.1%,10,4,2,1,True,False


In [54]:
#Nevermind, let's make sure the values in Pass Cmp are floats, and include the % in the name of the column.
data_full.rename(columns={'Pass Cmp': 'Pass Cmp %'}, inplace=True)
data_full['Pass Cmp %'] = data_full['Pass Cmp %'].astype(str).str.rstrip('%').astype(float)
data_full

,Date,TotalGoalsScored,FullTimeResult,Team,Shots,ShotsOnTarget,Pass Cmp %,FoulsCommited,Corners,YellowCardsEarned,RedCardsEarned,IsHome,Won
0,01/01/17,0,H,Crystal Palace,7,4,NaN,7,4,1,0,False,False
1,01/01/17,4,A,Tottenham,19,6,NaN,6,3,0,0,False,True
2,02/01/17,0,H,Southampton,12,3,NaN,14,8,1,0,False,False
3,02/01/17,1,H,Burnley,11,3,NaN,11,8,3,0,False,False
4,02/01/17,0,D,Leicester,10,4,NaN,9,2,0,0,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,08/12/18,0,A,Leicester,11,3,76.2,12,6,3,0,True,False
3500,08/12/18,4,H,Man United,20,11,83.4,11,10,1,0,True,True
3501,08/12/18,3,H,West Ham,13,6,78.1,10,5,1,0,True,True
3502,09/12/18,1,A,Newcastle,12,4,76.1,10,4,2,1,True,False


The formatting here is 'DD/MM/YY' (or sometimes DD/MM/YYYY, but we'll stick to the first). We'll make sure our managers data reflects that (this also makes it easier to convert to timestamps later on).

In [55]:
np.random.choice(managers_cleaned['Join date'].unique(), size=10)

array(['Jun 19, 2023', 'May 16, 2012', 'Dec 27, 2013', 'Feb 14, 2014',
       'Feb 26, 2019', 'Nov 11, 1998', 'Nov 11, 1998', 'Nov 27, 2007',
       'Dec 19, 2009', 'Dec 8, 2010'], dtype=object)

In [56]:
def clean_date(date):
    months = [None, 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    if not isinstance(date, str):
        return date
    date = date.split()
    month = months.index(date[0])
    day = date[1].strip(',')
    year = date[2]
    return f'{day}/{month}/{year}'
date = np.random.choice(managers_cleaned['Join date'].unique())
print(date)
clean_date(date)

Nov 26, 2007


'26/11/2007'

In [57]:
managers_cleaned['Join date'] = managers_cleaned['Join date'].apply(clean_date)
np.random.choice(managers_cleaned['Join date'].unique(), size=10)

array(['4/6/1993', '27/7/2020', '14/12/2013', '30/6/2014', '1/11/1994',
       '7/10/2001', '4/10/2015', '3/1/2017', '25/10/2017', '24/2/2019'],
      dtype=object)

#### Saving our work
After the data's been cleaned, we just save them to csvs to work with elsewhere. Cool!

In [58]:
data_full.to_csv('data/metrics_clean.csv', index=False)

In [60]:
managers_cleaned.to_csv('data/managers_clean.csv', index=False)
print(pd.__version__)

2.0.3


In [62]:
!python --version

Python 3.9.5
